In [ ]:
# Import pieces from codeflare-sdk
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication

In [ ]:
# Create authentication object and log in to desired user account (if not already authenticated)
auth = TokenAuthentication(
    token = "XXXX",
    server = "XXXX",
    skip_tls = False
)
auth.login()


NOTE: The default images used by the CodeFlare SDK for creating a RayCluster resource depend on the installed Python version:

- For Python 3.9: 'quay.io/modh/ray:2.35.0-py39-cu121'
- For Python 3.11: 'quay.io/modh/ray:2.35.0-py311-cu121'

If you prefer to use a custom Ray image that better suits your needs, you can specify it in the image field to override the default.

In [ ]:
# Create and submit our cluster
# The SDK will try to find the name of your default local queue based on the annotation "kueue.x-k8s.io/default-queue": "true" unless you specify the local queue manually below
cluster_name = "hfgputest-1"

cluster = Cluster(ClusterConfiguration(
                                       name=cluster_name,
                                       head_extended_resource_requests={'nvidia.com/gpu':0}, # For GPU enabled workloads set the head_extended_resource_requests and worker_extended_resource_requests
                                       worker_extended_resource_requests={'nvidia.com/gpu':0},
                                       num_workers=1,
                                       worker_cpu_requests=1,
                                       worker_cpu_limits=1,
                                       worker_memory_requests=4,
                                       worker_memory_limits=4,
                                       # image="", # Optional Field 
                                       write_to_file=False, # When enabled Ray Cluster yaml files are written to /HOME/.codeflare/resources 
                                       # local_queue="local-queue-name" # Specify the local queue manually
                                       ))

In [ ]:
cluster.up()

In [ ]:
cluster.wait_ready()

### Connect via the rayclient route

In [ ]:
from codeflare_sdk import generate_cert

generate_cert.generate_tls_cert(cluster_name, cluster.config.namespace)
generate_cert.export_env(cluster_name, cluster.config.namespace)

In [ ]:
import ray

ray.shutdown()
ray.init(address=cluster.local_client_url(), logging_level="DEBUG")

In [ ]:
import math
import ray

@ray.remote
def heavy_calculation_part(num_iterations):
    result = 0.0
    for i in range(num_iterations):
        for j in range(num_iterations):
            for k in range(num_iterations):
                result += math.sin(i) * math.cos(j) * math.tan(k)
    return result
@ray.remote
def heavy_calculation(num_iterations):
    results = ray.get([heavy_calculation_part.remote(num_iterations//30) for _ in range(30)])
    return sum(results)


In [ ]:
ref = heavy_calculation.remote(3000)

In [ ]:
ray.get(ref)

In [ ]:
ray.cancel(ref)
ray.shutdown()

In [ ]:
cluster.down()